# 순차 데이터(sequential data)

- 텍스트나 시계열 데이터(time series data)와 같이 순서에 의미가 있는 데이터
    - 예) "I am a boy"는 의미를 이해할 수 있지만 "boy am I a"는 의미를 이해할 수 없음
    - 일별 온도 데이터에서 날짜 순서가 섞인다면 내일의 온도를 쉽게 예상하기 어려움
    
- 감성분석에서는 이전에 입력된 데이터를 기억하는 기능이 중요할 수 있음
    - 예) "맛은 없지만 추천해요" 에서 "추천해요"를 해석할 때 앞의 "맛은 없지만"을 기억하고 있어야 무조건 긍정이라고 판단하지 않음
    - 퍼셉트론이나 CNN은 이런 기억장치가 없음
        - 하나의 배치를 사용하여 정방향 연산을 수행하고 나면 해당 샘플은 다음 샘플을 처리할 때 사용하지 않음
        - 이처럼 입력 데이터의 흐름이 앞으로만 전달되는 신경망을 피드포워드 신경망(feedforward neural network : FFNN)이라고 함
    - 신경망이 이전에 처리했던 샘플을 다음 샘플을 처리할 때 재사용하기 위해서는 데이터가 신경망 층에서 순환을 해야 함

# 순환 신경망(recurrent neural network : RNN)

<img src = "./image/rnn.png">

## 순환 신경망 구조

- 피드 포워드 신경망에 이전 데이터의 처리 흐름을 순환하는 고리 하나만 추가된 형태
    - 바로 이전에 사용했던 데이터를 재사용하는 구조

<img src = "./image/rnn_image1.png">

1. 첫 번재 샘플 x0를 처리하고 출력 h0가 다시 유닛으로 입력됨
    - h0에는 x0에 대한 정보가 포함되어 있음
    
2. x1을 처리할 때 h0를 함께 사용
    - h0와 x1을 사용해서 만든 h1에는 x0에 대한 정보가 어느 정도 포함되어 있음
    
3. x2를 처리할 때 h1을 함께 사용
    - 결과물 h2에는 x1과 x0에 대한 정보도 담겨 있음
    
- 위의 과정처럼 샘플을 처리하는 각 단계를 타임스텝(timestep)이라고 말함
    - 순환신경망은 이전 타임스텝의 샘플을 기억한다
    
- 타임스텝이 오래될수록 순환되는 정보는 희미해짐
- 순환신경망에서는 layer를 **셀(cell)** 이라고 부름
    - 하지만 입력에 가중치를 곱하고 활성화 함수를 통과시켜 다음 층으로 전달하는 기본 구조는 같음
    - 한 셀에는 여러 개의 유닛이 있지만 일반적으로 모든 유닛을 표시하지 않고 하나의 셀로 층을 표현함
    
- 셀의 출력은 은닉상태(hidden state)라고 부름

## 하이퍼볼릭 탄젠트 함수

<img src = "./image/tanh.png">

- 일반적으로 은닉층의 활성화 함수로는 하이퍼볼릭 탄젠트(hyperbolic tangent)함수인 tanh가 많이 활용됨
    - tanh함수는 -1 ~ 1 사이의 범위를 가짐

## 가중치 공유

<img src = "./image/rnn_weight.png">

- 순환 신경망도 피드포워드 신경망과 같이 입력값과 가중치를 곱하지만 이전 타입스텝의 은닉상태에 곱해지는 가중치가 하나 더 있음
- 모든 타입스텝에서 사용되는 가중치 W는 하나임
    - 가중치 W는 타임스텝에 따라 변화되는 뉴런의 출력을 학습
- 맨 처음 타임스텝1에서 사용되는 이전 은닉상태는 이전 타임스텝이 없기 때문에 0으로 초기화

## 파라미터 수

<img src = "./image/rnn_param.png">

- 각 유닛의 은닉상태가 다음 스텝에 재사용될 때 모든 유닛에 값이 전달됨
    - 즉, 이전 타입스텝의 은닉상태는 다음 타입스텝의 모든 유닛에 완전연결됨
- 순환 신경망의 파라미터 수 = (유닛 수 * 유닛 수) + (입력값의 수 * 유닛 수) + 유닛 수

# 입력

- 합성곱 층의 입력은 하나의 샘플이 3개의 차원을 가짐
    - 너비, 높이, 채널
    
- 순환층은 일반적으로 샘플마다 2개의 차원을 가짐
    - 하나의 샘플을 하나의 시퀀스(sequence)라고 말함
    - 하나의 시퀀스 안에는 여러 개의 아이템이 들어 있음
    - 시퀀스의 길이가 타임스텝의 길이가 됨
        - 예) 샘플 "I am a boy" 4개의 아이템으로 구성된 시퀀스
        - 각 아이템을 3개의 숫자로 표현한다면 타임스텝의 길이는 (1, 4, 3)
        
- 입력이 순환층을 통과하면 순환층의 유닛 개수만큼 출력됨
    - 하나의 샘플은 (단어의 개수, 단어 표현) 의 2차원 배열
    - 순환층을 통과하면 1차원 배열로 바뀜
        - 이 1차원 배열의 크기는 순환층의 유닛의 개수
        
    - 기본적으로 순환층은 마지막 타임스텝의 은닉 상태만 출력으로 내보냄
        - 입력된 시퀀스를 모두 읽어서 정보를 마지막 은닉 상태에 압축하여 전달
        
- 만약 순환층을 여러 개 쌓는다면
    - 순환층의 입력은 샘플마다 타임스텝과 단어 표현으로 이루어진 2차원 배열이어야 함
        - 첫 번째 셀이 마지막 타임스텝의 은닉상태만 출력한다면 입력 형식이 맞지 않음
        - 이 경우에는 마지막 셀을 제외한 모든 셀은 모든 타임스텝의 은닉 상태를 출력
        - 마지막 셀은 마지막 타임스텝의 은닉 상태만 출력

## 출력

- 순환신경망도 합성곱신경망과 마찬가지로 마지막에 밀집층을 두어 클래스를 분류
    - 다중 분류 : 출력층에 클래스 개수만큼 유닛을 두고 소프트맥스 출력층 함수 사용
    - 이진 분류 : 하나의 유닛을 두고 시그모이드 출력층 함수 사용
- 순환신경망의 차이점은 마지막 셀의 출력이 1차원이기 때문에 Flatten이 필요 없음